<a href="https://colab.research.google.com/github/B-lilily/Student-Performance-Analysis/blob/main/ml-app-gradio/3_Speech-2-text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Automatic Speech Recognition


What is speech to text?

- Automatic Speech Recognition (ASR), also known as Speech to Text (STT), is the task of transcribing a given audio to text.
[Reference](https://huggingface.co/tasks/automatic-speech-recognition)

##### Thonburian Whisper

https://huggingface.co/biodatlab/whisper-th-medium-combined

### Step 1: Install Transformers
Install the Hugging Face Transformers library so we can use the Whisper speech to text pipeline.


In [1]:
!pip install transformers

### Step 2: Load the ASR model and pipeline
Import Transformers and PyTorch, pick a Whisper model, choose CPU or GPU, and configure Thai transcription.


In [2]:
from transformers import pipeline
import torch

MODEL_NAME = "biodatlab/whisper-th-medium-combined"  # specify the model name
lang = "th"  # change to Thai langauge

device = 0 if torch.cuda.is_available() else "cpu"

pipe = pipeline(
    task="automatic-speech-recognition", # specify the task
    model=MODEL_NAME, # define the model
    chunk_length_s=30, # specify the chunk length
    device=device, # use GPU if available
)

pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(
  language=lang,
  task="transcribe" # specify the task
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


### Optional: Install pytubefix
Only needed if you want to download a YouTube audio sample.


In [3]:
!pip install pytubefix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 MB 12.5 MB/s eta 0:00:00


### Optional: Download a sample audio file
Fetch audio from YouTube, save it as an MP3 in `audio_example/`, and use the repo sample if the download fails.


In [14]:
# importing packages
from pytubefix import YouTube
import os

yt = YouTube("https://youtube.com/shorts/LEe7TgYSEbw?si=darObbc76rrxEsZ1")  #
## if you can't use this code, please use the example audio file in the repo

video = yt.streams.filter(only_audio=True).first()
os.makedirs("audio_example", exist_ok=True)
out_file = video.download(output_path="audio_example")

new_file = "audio_example/audio" + '.mp3'
os.rename(out_file, new_file)


### Step 3: Transcribe the audio file
Run the pipeline on the MP3 file. This can take a while on CPU.


In [15]:
text = pipe("audio_example/audio.mp3")["text"] # give audio mp3 and transcribe text
## this would take a while to process

### Step 4: View the transcription
Print the text output so you can inspect the result.


In [16]:
print(text)

กับแท็กซี่นี่คือยังไม่ได้เป็นนักเรื่อง ไม่ๆ ไม่เป็นน้องชายครับ ตอนนี้เป็นน้องชายก่อน ไม่ครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ น้องชายครับ


### Bonus: Build a Gradio demo
Create a small web UI that records microphone audio and returns the transcription.


In [17]:
import gradio as gr

def transcribe_audio(audio):
    return pipe(audio)["text"]

mic_input = gr.Audio(type="filepath", label="Speak into the microphone")
text_output = gr.Textbox(label="Transcribed Text")

demo = gr.Interface(
    fn=transcribe_audio,
    inputs=mic_input,
    outputs=text_output,
    title="Speech-to-Text Transcription",
    description="Speak into the microphone and get the transcribed text."
)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://87480913744b9d5d6b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
